In [1]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch

model_id = "stabilityai/stable-diffusion-2"

scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipeline = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipeline = pipeline.to('cuda:1')

/hpi/fs00/home/leon.hermann/mambaforge/envs/ethics/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.5.1+cu121 with CUDA 1201 (you have 2.5.1)
    Python  3.11.11 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
Loading pipeline components...: 100%|██████████| 6/6 [00:33<00:00,  5.58s/it]


In [ ]:
from src.utils import for_each_prompt
import os

def generate_images(folder:str, setting: str, key: str, obj:str, prefixes: list[str], images_per_prompt: int):
    prompts = []
    for prefix in prefixes:
        prompt = "A photo portrait of a " + prefix + " " + obj + " at " + setting
        prompts.append(prompt)
        
    output = pipeline(prompts, num_images_per_prompt=images_per_prompt)
    for i, prefix in enumerate(prefixes):
        starting_idx = i * images_per_prompt
        for i, image in enumerate(output.images[starting_idx:starting_idx + images_per_prompt]):
            output_folder = f"{folder}/{setting}/{key}/{obj}"
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
            image.save(f"{output_folder}/{prefix}_{i}.png")

for_each_prompt("prompts.json", "images", "work", generate_images)
for_each_prompt("prompts.json", "images", "home", generate_images)

100%|██████████| 50/50 [01:40<00:00,  2.01s/it]
